In [7]:
import cv2

def get_video_resolution(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Video file could not be opened.")
        return -1, -1

    res_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    res_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    cap.release()

    return res_h, res_w

# 비디오 파일 경로
humansc3d_train_50591643 = '../humansc3d/train/s01/videos/50591643/001.mp4'
humansc3d_train_58860488 = '../humansc3d/train/s01/videos/58860488/001.mp4'
humansc3d_train_60457274 = '../humansc3d/train/s01/videos/60457274/001.mp4'
humansc3d_train_65906101 = '../humansc3d/train/s01/videos/65906101/001.mp4'

humansc3d_val_50591643 = '../humansc3d/val/s06/videos/50591643/001.mp4'
humansc3d_val_58860488 = '../humansc3d/val/s06/videos/58860488/001.mp4'
humansc3d_val_60457274 = '../humansc3d/val/s06/videos/60457274/001.mp4'
humansc3d_val_65906101 = '../humansc3d/val/s06/videos/65906101/001.mp4'

fit3d_train_50591643 = '../fit3d/train/s03/videos/50591643/band_pull_apart.mp4'
fit3d_train_58860488 = '../fit3d/train/s03/videos/58860488/band_pull_apart.mp4'
fit3d_train_60457274 = '../fit3d/train/s03/videos/60457274/band_pull_apart.mp4'
fit3d_train_65906101 = '../fit3d/train/s03/videos/65906101/band_pull_apart.mp4'

fit3d_val_50591643 = '../fit3d/val/s09/videos/50591643/band_pull_apart.mp4'
fit3d_val_58860488 = '../fit3d/val/s09/videos/58860488/band_pull_apart.mp4'
fit3d_val_60457274 = '../fit3d/val/s09/videos/60457274/band_pull_apart.mp4'
fit3d_val_65906101 = '../fit3d/val/s09/videos/65906101/band_pull_apart.mp4'

path = {'humansc3d_train_50591643': humansc3d_train_50591643, 'humansc3d_train_58860488' : humansc3d_train_58860488,
        'humansc3d_train_60457274' : humansc3d_train_60457274, 'humansc3d_train_65906101' : humansc3d_train_65906101,
        'humansc3d_val_50591643' : humansc3d_val_50591643, 'humansc3d_val_58860488' : humansc3d_val_58860488,
        'humansc3d_val_60457274' : humansc3d_val_60457274, 'humansc3d_val_65906101' : humansc3d_val_65906101,
        'fit3d_train_50591643' : fit3d_train_50591643, 'fit3d_train_58860488' : fit3d_train_58860488,
        'fit3d_train_60457274' : fit3d_train_60457274, 'fit3d_train_65906101' : fit3d_train_65906101,
        'fit3d_val_50591643' : fit3d_val_50591643, 'fit3d_val_58860488' : fit3d_val_58860488,
        'fit3d_val_60457274' : fit3d_val_60457274, 'fit3d_val_65906101' : fit3d_val_65906101}

# 해상도 추출
for key, value in path.items():
    height, width = get_video_resolution(value)
    print(f"{key} ::: Height: {height}, Width: {width}")


humansc3d_train_50591643 ::: Height: 900, Width: 900
humansc3d_train_58860488 ::: Height: 900, Width: 900
humansc3d_train_60457274 ::: Height: 900, Width: 900
humansc3d_train_65906101 ::: Height: 900, Width: 900
humansc3d_val_50591643 ::: Height: 900, Width: 900
humansc3d_val_58860488 ::: Height: 900, Width: 900
humansc3d_val_60457274 ::: Height: 900, Width: 900
humansc3d_val_65906101 ::: Height: 900, Width: 900
fit3d_train_50591643 ::: Height: 900, Width: 900
fit3d_train_58860488 ::: Height: 900, Width: 900
fit3d_train_60457274 ::: Height: 900, Width: 900
fit3d_train_65906101 ::: Height: 900, Width: 900
fit3d_val_50591643 ::: Height: 900, Width: 900
fit3d_val_58860488 ::: Height: 900, Width: 900
fit3d_val_60457274 ::: Height: 900, Width: 900
fit3d_val_65906101 ::: Height: 900, Width: 900


In [46]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

In [47]:
fit3d_root = '../fit3d/train'
humansc3d_root = '../humansc3d/train
save_root = '../keypoints/train'

SyntaxError: EOL while scanning string literal (2213198298.py, line 2)

In [48]:
def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [49]:
def joint_format_to_h36m(joints):
    joints_crop = joints[:, :17, :]
    idx_pairs = [(11, 14), (12, 15), (13, 16)]
    
    for idx1, idx2 in idx_pairs:
        joints_crop[:, [idx1, idx2]] = joints_crop[:, [idx2, idx1]]

    return joints_crop

In [50]:
def read_cam_params(cam_path):
    with open(cam_path) as f:
        cam_params = json.load(f)
        for key1 in cam_params:
            for key2 in cam_params[key1]:
                cam_params[key1][key2] = np.array(cam_params[key1][key2])
    return cam_params

In [51]:
def project_3d_to_2d(joints3d, intrinsics):
    p = intrinsics['p'][:, [1, 0]]
    x = joints3d[:, :2] / joints3d[:, 2:3]
    r2 = np.sum(x**2, axis=1)
    radial = 1 + np.transpose(np.matmul(intrinsics['k'], np.array([r2, r2**2, r2**3])))
    tan = np.matmul(x, np.transpose(p))
    xx = x*(tan + radial) + r2[:, np.newaxis] * p
    proj = intrinsics['f'] * xx + intrinsics['c']
    return proj

In [52]:
def save_3d_npy(data_root, save_root):
    if not os.path.exists(data_root):
        raise FileNotFoundError(f"Data root directory not found: {data_root}")

    if not os.path.exists(save_root):
        os.makedirs(save_root)

    sub_folder_lst = [name for name in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, name))]
    
    for sub_folder in sub_folder_lst:
        json_path = os.path.join(data_root, sub_folder, 'joints3d_25')
        
        if not os.path.exists(json_path):
            print(f"Skipping missing directory: {json_path}")
            continue

        json_lst = [file for file in os.listdir(json_path) if file.endswith('.json')]
        
        for i, json_file in enumerate(tqdm(json_lst, desc=f'Processing {sub_folder}')):
            joint_file = os.path.join(json_path, json_file)
            if not os.path.isfile(joint_file):
                print(f"Skipping missing file: {joint_file}")
                continue

            joint_data = read_json(joint_file)
            joint_np = np.array(joint_data['joints3d_25'])
            joints = joint_format_to_h36m(joint_np)
            
            print(joints.shape)

#             file_path = os.path.join(save_root, f'{sub_folder}_sequence_{i}_3D.npy')
#             np.save(file_path, joints)

In [53]:
def save_2d_npy(data_root, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    subset = 'train'
    subjects = [name for name in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, name))]
    cameras = ['50591643', '58860488', '60457274', '65906101']

    for subj in subjects:
        actions_path = os.path.join(data_root, subset, subj, 'joints3d_25')
        actions = [action.split('.')[0] for action in os.listdir(actions_path) if action.endswith('.json')]
        for action_index, action in enumerate(tqdm(actions, desc='Saving 2D keypoints')):
            for camera_index, cam in enumerate(cameras):
                cam_path = os.path.join(data_root, subset, subj, 'camera_parameters', cam, f'{action}.json')
                j3d_path = os.path.join(data_root, subset, subj, 'joints3d_25', f'{action}.json')

                if not os.path.exists(cam_path) or not os.path.exists(j3d_path):
                    continue

                cam_params = read_cam_params(cam_path)
                with open(j3d_path) as f:
                    j3ds = np.array(json.load(f)['joints3d_25'])

                all_frames_2d = np.zeros((len(j3ds), 25, 2))
                for frame_id, j3d in enumerate(j3ds):
                    j2d = project_3d_to_2d(j3d, cam_params['intrinsics'])
                    all_frames_2d[frame_id] = j2d

                # H36M 포맷으로 변경
                all_frames_2d = joint_format_to_h36m(all_frames_2d)

                # Confidence score 추가
                confidence_scores = np.ones((all_frames_2d.shape[0], all_frames_2d.shape[1], 1))
                all_frames_2d_with_confidence = np.concatenate((all_frames_2d, confidence_scores), axis=-1)

                output_path = os.path.join(output_dir, f'fit3d_sequence_{action_index + 1}_{camera_index + 1}_2D.npy')
                np.save(output_path, all_frames_2d_with_confidence)

In [ ]:
save_3d_npy(humansc3d_root, save_root)

In [54]:
cam_params = read_cam_params('../fit3human/train/S01/camera_parameters/50591643/001.json')

In [57]:
cam_params.keys()

dict_keys(['extrinsics', 'intrinsics_w_distortion', 'intrinsics_wo_distortion'])

In [60]:
cam_params['extrinsics']['T'].shape

(1, 3)

In [61]:
cam_params

{'extrinsics': {'R': array([[-0.40123066, -0.91513736,  0.03921192],
         [-0.19776205,  0.04474824, -0.97922815],
         [ 0.89437359, -0.40065099, -0.19893381]]),
  'T': array([[-3.93689187,  1.94731118,  1.54806366]])},
 'intrinsics_w_distortion': {'f': array([[1098.92904454, 1095.92833681]]),
  'c': array([[463.93264039, 447.62489594]]),
  'k': array([[-0.19654847,  0.10420355,  0.00410303]]),
  'p': array([[-0.00045064,  0.00112772]])},
 'intrinsics_wo_distortion': {'c': array([463.94235229, 447.64175415]),
  'f': array([1096.38378906, 1090.24389648])}}

In [73]:
cam_params['intrinsics_w_distortion']['p'].shape

(1, 2)

In [64]:
import json
def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [77]:
path = '../fit3human/train/S01/joints3d_25/001.json'
j3d = read_json(path)
j3d_np = np.array(j3d['joints3d_25'])

In [82]:
j3d_np[:,:,:2].shape

(354, 25, 2)

In [89]:
# 254 x 17 x 3 형태의 배열 생성하고, 2차원 좌표를 복사한 후 z값을 1로 설정
array_3d = np.ones(j3d_np.shape)  # 모든 값을 1로 초기화
array_3d[:, :, :2] = j3d_np[:,:,:2]

In [90]:
array_3d

array([[[ 0.0285847,  0.119295 ,  1.       ],
        [ 0.0399862,  0.251669 ,  1.       ],
        [ 0.0237128,  0.2429   ,  1.       ],
        ...,
        [ 0.170764 ,  0.429925 ,  1.       ],
        [ 0.167963 , -0.141363 ,  1.       ],
        [ 0.142642 , -0.228158 ,  1.       ]],

       [[ 0.0287701,  0.119362 ,  1.       ],
        [ 0.0401995,  0.251732 ,  1.       ],
        [ 0.0238572,  0.24293  ,  1.       ],
        ...,
        [ 0.171043 ,  0.429779 ,  1.       ],
        [ 0.168129 , -0.141836 ,  1.       ],
        [ 0.142807 , -0.22837  ,  1.       ]],

       [[ 0.0289465,  0.119448 ,  1.       ],
        [ 0.0403863,  0.251815 ,  1.       ],
        [ 0.024038 ,  0.242967 ,  1.       ],
        ...,
        [ 0.171404 ,  0.429697 ,  1.       ],
        [ 0.1683   , -0.142066 ,  1.       ],
        [ 0.142885 , -0.228559 ,  1.       ]],

       ...,

       [[ 0.0343252,  0.121446 ,  1.       ],
        [ 0.0459674,  0.253835 ,  1.       ],
        [ 0.027177 ,  

NameError: name 'FitHscDataset3D' is not defined

In [32]:
47*3*4

564

In [37]:
172*3

516

In [38]:
len([name for name in os.listdir('../keypoints/train') if os.path.isdir(os.path.join('../keypoints/train', name))])

0

In [44]:
len([action.split('.')[0] for action in os.listdir('../keypoints/train') if action.endswith('.npy')])

6008

In [42]:
(564+688)*2

2504

In [91]:
import numpy as np
data_3d = np.load('../keypoints/train/sequence_100_3D.npy')
data_2d = np.load('../keypoints/train/sequence_100_2D.npy')

In [32]:
root_joint = np.array([-0.0426111, 0.170725, 0.913295])

In [33]:
tl_joint = np.array([-0.0426111, 0.170725, 0.913295])
br_joint = np.array([-0.0426111, 0.170725, 0.913295])

tl_joint[:2] -= 1000;tl_joint
br_joint[:2] -= 1000;br_joint

array([-1.00004261e+03, -9.99829275e+02,  9.13295000e-01])

In [36]:
data_2d.shape

(226, 17, 3)

In [43]:
data_2d[:,:,2].max()

1.0

In [94]:
np.min(data_3d)

-92.17922591442976

In [95]:
data_3d[0]

array([[ 3.99570650e+02, -9.17661183e+00,  0.00000000e+00],
       [ 4.18977523e+02,  6.14266511e+00, -2.86230753e-04],
       [ 3.77158641e+02,  5.53120847e+01, -1.11931529e-04],
       [ 3.30084346e+02,  1.20986303e+02,  3.50068974e-05],
       [ 3.80025919e+02, -2.41102920e+01,  2.87550528e-04],
       [ 3.35163069e+02,  2.15552276e+01,  4.87339058e-04],
       [ 2.80669675e+02,  8.15747370e+01,  7.21179155e-04],
       [ 4.20440766e+02, -3.17210996e+01, -9.48535891e-05],
       [ 4.46600545e+02, -5.21001057e+01, -2.65017317e-04],
       [ 4.58378124e+02, -6.82547494e+01, -3.07937698e-04],
       [ 4.63259524e+02, -6.78900021e+01, -3.60612810e-04],
       [ 4.19389331e+02, -5.98104027e+01,  7.80673980e-05],
       [ 3.79266085e+02, -5.48338413e+01,  4.67170495e-04],
       [ 3.57441432e+02, -5.48476033e+01,  6.54796968e-04],
       [ 4.60825125e+02, -3.02730317e+01, -5.42014933e-04],
       [ 4.52901060e+02,  3.85661754e+00, -6.35684790e-04],
       [ 4.41746837e+02,  1.92050114e+01

In [15]:
data_2d

array([[[4.06872734e+02, 6.19146759e+02, 1.00000000e+00],
        [4.12434448e+02, 7.68991106e+02, 1.00000000e+00],
        [3.62214253e+02, 1.04643387e+03, 1.00000000e+00],
        ...,
        [4.06780685e+02, 6.94799807e+02, 1.00000000e+00],
        [4.08812666e+02, 8.36364350e+02, 1.00000000e+00],
        [5.04518878e+02, 9.60438882e+02, 1.00000000e+00]],

       [[4.06210242e+02, 6.19338396e+02, 1.00000000e+00],
        [4.11545832e+02, 7.69183110e+02, 1.00000000e+00],
        [3.61761729e+02, 1.04654996e+03, 1.00000000e+00],
        ...,
        [4.06694729e+02, 6.94977070e+02, 1.00000000e+00],
        [4.08108801e+02, 8.36473790e+02, 1.00000000e+00],
        [5.02779350e+02, 9.60758480e+02, 1.00000000e+00]],

       [[4.05748607e+02, 6.19531831e+02, 1.00000000e+00],
        [4.10675689e+02, 7.69393519e+02, 1.00000000e+00],
        [3.61743749e+02, 1.04630756e+03, 1.00000000e+00],
        ...,
        [4.06634652e+02, 6.95069743e+02, 1.00000000e+00],
        [4.07344759e+02, 8.36

In [18]:
np.min(data_2d)

-167778384.48020098

In [19]:
np.max(data_2d)

352449190.5205567

In [25]:
data_np = np.random.rand(243, 17, 3)
data = np.zeros_like(data_np)
for idx, frame in enumerate(data):
    add = np.array([1,1,1])
    data[idx] = add
data

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]])

In [22]:
data_np[0]

array([[3.41086310e-02, 8.07247187e-01, 3.70000493e-01],
       [8.73030871e-01, 1.95725620e-01, 5.42902961e-01],
       [6.50396147e-01, 5.80490505e-02, 6.66372736e-01],
       [3.41833583e-01, 5.56208982e-01, 1.95303397e-02],
       [4.26217632e-01, 1.99433086e-01, 7.06356505e-01],
       [6.73847204e-01, 8.76644840e-01, 2.69465785e-01],
       [4.49628964e-01, 1.41309592e-02, 3.54222155e-02],
       [2.18800975e-01, 7.78031412e-03, 9.73438014e-01],
       [4.24484422e-01, 6.07890840e-01, 7.82625629e-01],
       [3.15955298e-02, 5.59159180e-01, 8.04391902e-02],
       [3.77930309e-01, 9.61400320e-01, 5.63913122e-02],
       [3.92691211e-01, 8.65595349e-01, 6.89526251e-04],
       [4.18727779e-01, 5.47951071e-01, 7.90201308e-02],
       [5.98066799e-01, 7.47652461e-01, 1.81199207e-01],
       [7.27966177e-01, 4.52394697e-01, 5.63330293e-01],
       [5.76688467e-01, 1.97746639e-01, 2.90129841e-01],
       [6.09717768e-01, 5.49163152e-02, 2.14429951e-01]])